# Labsheet 3: Motors

This worksheet will cover getting your motors to move on your Romi Robot.  In previous labsheets you will have:
 - Seen the standard Arduino sketch, which has the structure of setup() and loop() routines.
 - Experimented with Arduino Example sketches
 - Uploaded sketches to your Romi robot.
 - Experimented with Serial commands to view debug output from your code.
 - Experimented with millis(), micros() and running blocks of code at different time intervals.
 
In this labsheet we are going to:

- Get the Romi Motors moving, and experiment with doing specified movements.


**Note:** The methods we develop in this labsheet are quite basic, and we will implement better methods in future labsheets.  However, this labsheet is important to introduce you to the basic concepts.  For now, do not worry about "perfect" performance - instead, try to understand the performance you see.

**Note:** We will look at building a class in Labsheet 5.  It is highly recommended you later implement a class for operating your motors.




# Exercise 1: Making Motors Move

Compared to a micro-controller, motors are typically high powered devices and require their own electronics to operate.  A device like an Arduino cannot normally power a motor directly.  Most microcontrollers will output 5volts at 25milliamps on a digital pin.  It is also the case that when we run a motor, the electronics can become unstable causing unpredictable computing performance.  On your Romi, you will likely see your robot reseting when the batteries get low and the motors are activated.  Therefore, motors are often kept isolated with their own electronics and power supplies.


Fortunately it is very convenient to buy modular pieces of electronics to drive a motor, normally referred to as **motor drivers**.  The Romi motors are DC motors. The motor drivers are built into the black circuit board of the Romi.  If you're interested, you might want to google <a href="https://lmgtfy.com/?q=h-bridge+circuit">H-Bridge drivers</a>, which is the configuration of electronics ordinarily used to drive a motor in two directions.  

It has become very common to operate a DC Motor Driver with two signals:
- Direction (DIR) - which way to turn.
- Pulse Width Modulation (PWM) - how much power, related to speed.

These two signals allow us to control the direction and the power sent to a motor.  The **motor driver** therefore has DIR and PWM as two inputs.  The DIR signal is digital, meaning high (5v) or low (0v), and will correspond to rotating the motor clockwise or counter-clockwise.  The motor driver PWM pin reads an analog voltage received - therefore a voltage in the range of 0v and 5v - and proportionally drives power to the motor.  It is **not safe** to assume the relationship between the PWM signal and the motor speed is linear - you should always check the datasheet and experiment with your motors.

Therefore, in order to drive your motor, you need to:
- Set the DIR pin to determine the direction of motion.
- Set the PWM pin to an appropriate voltage level to determine the speed of rotation.

An Arduino (or other microcontroller) can generate a PWM signal as an output on a digital pin.  This is used to synthesise an approximation to an analog signal.  With Arduino, a function called _analogWrite()_ performs this operation.  Writing a value of 255 creates a constant high signal (5v), whilst writing a 0 creates a constant low signal (0v).  Values between 0 and 255 generate square waveforms with a ratio of _mark_ to _space_ (high, low), which can be thought of as _averaging_ out to a voltage between 0v and 5v when received by another device.  The more time the square waveform is high, the closer the voltage is to 5v, and vice-versa.

<a href="https://www.arduino.cc/en/uploads/Tutorial/pwm.gif"><img src="PWM.gif"></a>

We can see that this use of PWM and DIR is common to motor drivers such as the <a href="https://www.active-robots.com/controllers/motor/dc-motor-controllers/pololu-g2-high-power-motor-driver-18v25.html"> Pololu G2</a> or the <a href="https://coolcomponents.co.uk/products/drv8835-dual-motor-driver-shield-for-arduino">DRV8835 Dual Motor Driver Shield for Arduino</a>.  Sometimes more advanced boards will have extra pins for motor braking, current control and current sensing.  Other motor drivers have advanced communication protocols like I2C, TWI or SPI.


## Using the Romi Motor Drivers

The documentation for the Romi is all <a href="https://www.pololu.com/product/3544/resources">available online</a>.  <a href="https://www.pololu.com/docs/0J69/3.8">Section 3.8</a> describes how all the pins of the board are used.  The following describes the connections to the motor driver pins:


![alt text](MotorDriverPins.png)

From this table we can read that:
- Left Motor PWM maps to Arduino Digital pin 10
- Left Motor DIR maps to Arduino Digital pin 16
- Right Motor PWM maps to Arduino Digital pin 9
- Right Motor DIR  maps to Arduino Digital pin 15

From this, we can write a very simple piece of code to test our  motors:
<pre>

<font color="#434f54">&#47;&#47; Pin definitions, to make the</font>
<font color="#434f54">&#47;&#47; code easier to read.</font>
<font color="#5e6d03">#define</font> <font color="#000000">L_PWM_PIN</font> <font color="#000000">10</font>
<font color="#5e6d03">#define</font> <font color="#000000">L_DIR_PIN</font> <font color="#000000">16</font>
<font color="#5e6d03">#define</font> <font color="#000000">R_PWM_PIN</font> &nbsp;<font color="#000000">9</font>
<font color="#5e6d03">#define</font> <font color="#000000">R_DIR_PIN</font> <font color="#000000">15</font>

<font color="#434f54">&#47;&#47; Variables to remember our</font>
<font color="#434f54">&#47;&#47; motor power for Left and Right.</font>
<font color="#434f54">&#47;&#47; Byte stores 0 to 255</font>
<font color="#00979c">byte</font> <font color="#000000">l_power</font><font color="#000000">;</font>
<font color="#00979c">byte</font> <font color="#000000">r_power</font><font color="#000000">;</font>

<font color="#434f54">&#47;&#47; Setup, only runs once when powered on.</font>
<font color="#00979c">void</font> <font color="#5e6d03">setup</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>

 &nbsp;<font color="#434f54">&#47;&#47; Set our motor driver pins as outputs.</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font> <font color="#000000">L_PWM_PIN</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font> <font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font> <font color="#000000">L_DIR_PIN</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font> <font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font> <font color="#000000">R_PWM_PIN</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font> <font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">pinMode</font><font color="#000000">(</font> <font color="#000000">R_DIR_PIN</font><font color="#434f54">,</font> <font color="#00979c">OUTPUT</font> <font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;<font color="#434f54">&#47;&#47; Set initial direction for l and r</font>
 &nbsp;<font color="#434f54">&#47;&#47; Which of these is foward, or backward?</font>
 &nbsp;<font color="#d35400">digitalWrite</font><font color="#000000">(</font> <font color="#000000">L_DIR_PIN</font><font color="#434f54">,</font> <font color="#00979c">LOW</font> &nbsp;<font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">digitalWrite</font><font color="#000000">(</font> <font color="#000000">R_DIR_PIN</font><font color="#434f54">,</font> <font color="#00979c">HIGH</font> <font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;
 &nbsp;<font color="#434f54">&#47;&#47; Set initial l_power and r_power values.</font>
 &nbsp;<font color="#000000">l_power</font> <font color="#434f54">=</font> <font color="#000000">0</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">r_power</font> <font color="#434f54">=</font> <font color="#000000">0</font><font color="#000000">;</font>

 &nbsp;<font color="#434f54">&#47;&#47; Start up the Serial for debugging.</font>
 &nbsp;<b><font color="#d35400">Serial</font></b><font color="#434f54">.</font><font color="#d35400">begin</font><font color="#000000">(</font><font color="#000000">9600</font><font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<b><font color="#d35400">delay</font></b><font color="#000000">(</font><font color="#000000">1000</font><font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#434f54">&#47;&#47; Print reset so we can catch any reset error.</font>
 &nbsp;<b><font color="#d35400">Serial</font></b><font color="#434f54">.</font><font color="#d35400">println</font><font color="#000000">(</font><font color="#005c5f">&#34; ***Reset*** &#34;</font><font color="#000000">)</font><font color="#000000">;</font>

<font color="#000000">}</font>


<font color="#434f54">&#47;&#47; put your main code here, to run repeatedly:</font>
<font color="#00979c">void</font> <font color="#5e6d03">loop</font><font color="#000000">(</font><font color="#000000">)</font> <font color="#000000">{</font>

 &nbsp;<font color="#434f54">&#47;&#47; Adjust power. e.g., increment by 4 on every loop()</font>
 &nbsp;<font color="#000000">l_power</font> <font color="#434f54">=</font> <font color="#000000">l_power</font> <font color="#434f54">+</font> <font color="#000000">4</font><font color="#000000">;</font>
 &nbsp;<font color="#000000">r_power</font> <font color="#434f54">=</font> <font color="#000000">r_power</font> <font color="#434f54">+</font> <font color="#000000">4</font><font color="#000000">;</font>

 &nbsp;<font color="#434f54">&#47;&#47; Send power PWM to pins, to motor drivers.</font>
 &nbsp;<font color="#d35400">analogWrite</font><font color="#000000">(</font> <font color="#000000">L_PWM_PIN</font><font color="#434f54">,</font> <font color="#000000">l_power</font> <font color="#000000">)</font><font color="#000000">;</font>
 &nbsp;<font color="#d35400">analogWrite</font><font color="#000000">(</font> <font color="#000000">R_PWM_PIN</font><font color="#434f54">,</font> <font color="#000000">r_power</font> <font color="#000000">)</font><font color="#000000">;</font>

 &nbsp;<font color="#434f54">&#47;&#47; Brief pause</font>
 &nbsp;<font color="#d35400">delay</font><font color="#000000">(</font><font color="#000000">250</font><font color="#000000">)</font><font color="#000000">;</font>

<font color="#000000">}</font>

</pre>

If you run the above code, you should notice that your motors both start **off** (not moving), and then steadily increase their speed toward maximum. Eventually, both suddenly revert to zero speed, and then increase their speed to maximum again.  





From the above example code:
- Are both motors rotating in the same direction?
    - Make a note of whether L_DIR_PIN and R_DIR_PIN need to be set HIGH or LOW to move your Romi forwards.
    - You may want to create a new #define statement to make your code easier to read, e.g. #define L_DIR_FWD HIGH


- Discuss with your peers: why do the motors suddenly change from full speed to zero speed?  


- Adjust the code so that the motors increase their speed to maximum, and then slowly back down to minimum, and repeats.  Try to use a millis() code-block (non-blocking) rather than a _for()_ or _while()_ loop (blocking).
 
 
# Exercise 2:

- **Experimental Observations:** the following observations will be _unique to your Romi_ and will be useful for future labsheets:
    - Experiment to find the **minimum** motor power required for your Romi to move fowards, note this down.
    - Experiment to find the **minimum** motor power required for your Romi to move backwards, note this down.
    - Are the above measurements the same values for forwards and backwards?
    - Are the above measurements the same for left and right wheels?
    - What factors (robot or environment) may influence your above observations?
    - The forward and reverse power values and that cannot produce movement are called a **deadband**, and are a difficult problem to deal with when controlling motors.  What is the range of values for your Romi that produce no movement?
    - Experiment to find the **maximum** motor power where there is no discernable difference in high speed.  How might you work this out experimentally?
    - Is it more important to conduct these experiments with your robot lifted from the surface (wheels rotating in free air) or on the surface?
    - See if you can build up a set of data to actually measure the speed of the robot.  You might need a long marker of distance on your work surface so that you can measure a time interval accurately.
        - speed = distance / time
    - Does the power sent to your motors have a linear relationship to the speed produced?


# Exercise 3:

Often when we work on robotic systems, we have to test our assumptions about how code is functioning.  Whilst the reference manual should tell us what software will do, we usually do not know how hardware may respond.  **Therefore, we must experiment and observe performance to be sure.**

- Change the above code so that `l_power` and `r_power` are of type **float**.  Note down the following observations:
    - What happens if you send negative numbers to _analogWrite()_?
    - What happens if a value larger than 255 is sent to _analogWrite()_?
    - **There are no 'correct' answers to these questions.**  
        - Conduct experiments to increase your understanding of the full behaviour of _analogWrite()_.  This is someone elses' code, so make sure you understand how it works and all the error conditions.
        - When we write software for robotic systems, the only way to fully understand the performance is to experiment and test your system.
        

# Exercise 4:

- Write a function for each motor that will take in a power as a signed value (positive and negative), and use the sign to set the direction for the motor.
- Write in code to catch error conditions.

    
# Exercise 5:

**For these exercises, do not attempt to achieve perfect peformance.  Instead, explore the performance you observe until you are happy that you understand the causes sufficiently.**
- Extend the above code to make the motors move through a sequence of power values.  Try to avoid using *for()*, *while()*, or other blocking iterators within the _loop()_ function - you may need to look back at Labsheet 2 which covered the use of _millis()_ to determine behaviour based on a period of ellapsed time.
    - Get your Romi robot to drive in a straight line for a fixed period of time:
        - How reliably does it drive in a straight line?
        - Create a method to measure the deviation from a straight line path over a fixed distance.
        - Experiment to see how reliably you can compensate by adjusting the power values of your motors.
     
    - Using time, attempt to get your Romi to drive forwards, then backwards, and stop where it started:
        - How reliably can your Romi get back to where it started?
        - Determine the margin of error by conducting repeated tests.  Find a +/- mm estimate for the distance achieved away from its starting position. 
    - Can you get your Romi to rotate on the spot by 90 degrees?
        - How reliably can it rotate a fixed angle?
    - Can you get your Romi to drive the outline of a square?